In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pyspark

     |████████████████████████████████| 204.7MB 56kB/s 
     |████████████████████████████████| 204kB 41.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=2418661a6f880266aacb97f937224c4a8a5aca60513e83dfdef9589cbf423a3d
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Customers').getOrCreate()

from pyspark.ml.regression import LinearRegression

In [6]:
df = spark.read.csv("/content/drive/My Drive/2020/Pyspark/Ecommerce_Customers.csv", inferSchema=True, header=True)

In [7]:
df

DataFrame[Email: string, Address: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double]

In [9]:
df.show(5)

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [12]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



* In sklearn
1. X1,X2,X3,X4,X5    Y1 ---->model-->prediction

* In Pyspark
2. [X1,X2,X3,X4,X5]  Y1---->model--->prediction

In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
features = VectorAssembler(inputCols=["Avg Session Length","Time on App","Time on Website","Length of Membership"], outputCol="Independent Features")

In [17]:
output = features.transform(df)

In [22]:
output.show(5)

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|Independent Features|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|[34.49726773,12.6...|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|[31.92627203,11.1...|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|[33.00091476,11.3...|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.7

In [23]:
output.select("Independent Features").show(5)

+--------------------+
|Independent Features|
+--------------------+
|[34.49726773,12.6...|
|[31.92627203,11.1...|
|[33.00091476,11.3...|
|[34.30555663,13.7...|
|[33.33067252,12.7...|
+--------------------+
only showing top 5 rows



In [24]:
output.columns

['Email',
 'Address',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent',
 'Independent Features']

In [25]:
finalized_data = output.select("Independent Features","Yearly Amount Spent")

In [26]:
finalized_data.show(5)

+--------------------+-------------------+
|Independent Features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.49726773,12.6...|         587.951054|
|[31.92627203,11.1...|        392.2049334|
|[33.00091476,11.3...|        487.5475049|
|[34.30555663,13.7...|         581.852344|
|[33.33067252,12.7...|         599.406092|
+--------------------+-------------------+
only showing top 5 rows



In [36]:
train, test = finalized_data.randomSplit([0.75,0.25])

In [37]:
lr_clf = LinearRegression(featuresCol='Independent Features', labelCol='Yearly Amount Spent')
lr = lr_clf.fit(train)

In [38]:
lr.coefficients

DenseVector([26.2318, 38.6105, 0.4531, 61.2711])

In [39]:
lr.intercept

-1066.2377058319769

In [40]:
pred = lr.evaluate(test)

In [41]:
pred.predictions.show()

+--------------------+-------------------+------------------+
|Independent Features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[29.53242897,10.9...|        408.6403511|396.55620951024275|
|[30.57436368,11.3...|        442.0644138|440.74004560385447|
|[30.83643267,13.1...|        467.5019004|470.69894584250255|
|[31.06132516,12.3...|        487.5554581| 492.6361760855491|
|[31.30919264,11.9...|        432.7207178| 429.1497926971358|
|[31.36621217,11.1...|        430.5888826| 426.1976394959745|
|[31.38958548,10.9...|        410.0696111| 409.0017597309027|
|[31.52575242,11.3...|        443.9656268|448.88611069532794|
|[31.6005122,12.22...|        479.1728515| 460.5306342747979|
|[31.72165236,11.7...|        347.7769266|349.59485663539135|
|[31.80930032,11.6...|        536.7718994| 547.0910677966149|
|[31.8530748,12.14...|        459.2851235| 461.3136190907121|
|[31.86274111,14.0...|        556.2981412| 557.4858442931425|
|[31.874